%tensorflow_version 2.x

In [35]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [36]:
dftrain=pd.read_csv(r'/test.csv')
dfeval=pd.read_csv(r'/train.csv')
dftrain.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [42]:
dftrain.isna().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0
Cabin,0


In [37]:
dftrain['Age'].fillna(dftrain['Age'].mean(),inplace=True)
dftrain.dropna(subset=['Fare'],inplace=True)
dftrain.fillna('unknown',inplace=True)

<ipython-input-37-9f61f760b167>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dftrain['Age'].fillna(dftrain['Age'].mean(),inplace=True)


In [38]:
dftrain.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000
mean,1100.635492,2.263789,30.200103,0.448441,0.393285,35.627188
std,120.923774,0.842077,12.562386,0.897568,0.982419,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.000000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1101.000000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1205.000000,3.000000,35.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [41]:
dfeval.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [40]:
dfeval['Age'].fillna(dfeval['Age'].mean(),inplace=True)
dfeval['Cabin'].fillna('unknown',inplace=True)
dfeval['Embarked'].fillna(method='ffill',inplace=True)

<ipython-input-40-d96f9ae16975>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfeval['Age'].fillna(dfeval['Age'].mean(),inplace=True)
<ipython-input-40-d96f9ae16975>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [43]:
print(dftrain.shape)
print(dfeval.shape)

(417, 11)
(891, 12)


In [44]:
y_eval=dfeval['Survived'].iloc[-200:].copy()
y_test=dfeval['Survived'].iloc[:-200].copy()
print(y_eval)
print(y_test)

691    1
692    1
693    0
694    0
695    0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 200, dtype: int64
0      0
1      1
2      1
3      1
4      0
      ..
686    0
687    0
688    0
689    1
690    1
Name: Survived, Length: 691, dtype: int64


In [46]:
CATEGORICAL_COLUMNS=['Name','Sex','Ticket','Cabin','Embarked']
NUMERICAL_COLUMNS=['PassengerId','Pclass','Age','SibSp','Parch','Fare']
feature_columns=[]
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary=dfeval[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))
for feature_name in NUMERICAL_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))



In [47]:
def make_input_fn(data_df,label_df,num_epochs=19,shuffle=True,batch_size=32):
  def input_funtion():
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds=ds.shuffle(1000)
    ds=ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_funtion
train_input_fn=make_input_fn(dfeval.drop('Survived', axis=1).iloc[:-200].copy(),y_test)
eval_input_fn=make_input_fn(dfeval.drop('Survived', axis=1).iloc[-200:].copy(),y_eval,num_epochs=1,shuffle=False)


In [48]:
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result=linear_est.evaluate(eval_input_fn)
clear_output()
print(result['accuracy'])

0.82
